In [1]:
import time
import requests
import numpy as np
import pandas as pd
from scrapy.http import TextResponse
import re

# Problem 1

In [2]:
def scraping_books(url,base_url="http://books.toscrape.com/"):
    page = requests.get(url)
    response = TextResponse(body=page.text,url=url,encoding="utf-8")
    title = response.css("h3 > a::attr(title)").extract()
    price1 = response.css("p.price_color::text").extract()
    price = [float(i.replace("Â£", "")) for i in price1]
    burl = response.css("h3 >a::attr(href)").extract()
    purl = response.css("img::attr(src)").extract()
    star = response.css("p[class^='star-rating']::attr(class)").extract()
    star_rating = []
    for i in star:
        star_rating.append(i.replace("star-rating", ""))
    stock  =response.css("p.price_color ~ p[class^='instock']::attr(class)").extract()
    instock = [i.replace("availability", " ") for i in stock]    
    base_url = "http://books.toscrape.com/catalogue/"
    bookurl = [base_url + i for i in burl]
    picurl = [base_url + i for i in purl]
    genere = []
    bookinfo = []
    for i in bookurl:
        page = requests.get(i)
        response = TextResponse(body=page.text,url=i,encoding="utf-8")
        genere.append(response.css("li~li~li > a::text")[0].extract())
        bookinfo.append(response.css("article[class='product_page'] > p::text").extract_first())
    
    return pd.DataFrame({"title":title,  "price":price, "bookurl":bookurl, "picurl":picurl,"star_rating":star_rating,"instock":instock,'genere':genere,"bookinfo":bookinfo})

In [3]:
books = []
for i in range(1,100):
    cureent_page =scraping_books(url = f"http://books.toscrape.com/catalogue/page-{i}.html")
    if cureent_page.shape[0] == 0:
        break
    else:
        books.append(cureent_page)

In [4]:
books = pd.concat(books)
books

,title,price,bookurl,picurl,star_rating,instock,genere,bookinfo
0,A Light in the Attic,51.77,http://books.toscrape.com/catalogue/a-light-in...,http://books.toscrape.com/catalogue/../media/c...,Three,instock,Poetry,It's hard to imagine a world without A Light i...
1,Tipping the Velvet,53.74,http://books.toscrape.com/catalogue/tipping-th...,http://books.toscrape.com/catalogue/../media/c...,One,instock,Historical Fiction,"""Erotic and absorbing...Written with starling ..."
2,Soumission,50.10,http://books.toscrape.com/catalogue/soumission...,http://books.toscrape.com/catalogue/../media/c...,One,instock,Fiction,"Dans une France assez proche de la nÃ´tre, un ..."
3,Sharp Objects,47.82,http://books.toscrape.com/catalogue/sharp-obje...,http://books.toscrape.com/catalogue/../media/c...,Four,instock,Mystery,"WICKED above her hipbone, GIRL across her hear..."
4,Sapiens: A Brief History of Humankind,54.23,http://books.toscrape.com/catalogue/sapiens-a-...,http://books.toscrape.com/catalogue/../media/c...,Five,instock,History,From a renowned historian comes a groundbreaki...
...,...,...,...,...,...,...,...,...
15,Alice in Wonderland (Alice's Adventures in Won...,55.53,http://books.toscrape.com/catalogue/alice-in-w...,http://books.toscrape.com/catalogue/../media/c...,One,instock,Classics,None
16,"Ajin: Demi-Human, Volume 1 (Ajin: Demi-Human #1)",57.06,http://books.toscrape.com/catalogue/ajin-demi-...,http://books.toscrape.com/catalogue/../media/c...,Four,instock,Sequential Art,High school student Kei Nagai is struck dead i...
17,A Spy's Devotion (The Regency Spies of London #1),16.97,http://books.toscrape.com/catalogue/a-spys-dev...,http://books.toscrape.com/catalogue/../media/c...,Five,instock,Historical Fiction,"In Englandâs Regency era, manners and elegan..."
18,1st to Die (Women's Murder Club #1),53.98,http://books.toscrape.com/catalogue/1st-to-die...,http://books.toscrape.com/catalogue/../media/c...,One,instock,Mystery,"James Patterson, bestselling author of the Ale..."


In [5]:
books.to_csv('books.csv', index=False)

In [6]:
books['price'].max()

59.99

#### most expensive book's price is 59.99

In [7]:
books[books['price'] == books['price'].max()]['genere']

8    Romance
Name: genere, dtype: object

#### most expensive book's genere is Romance

In [8]:
books['star_rating'].value_counts()

 One      226
 Three    203
 Five     196
 Two      196
 Four     179
Name: star_rating, dtype: int64

#### count of books by star rating

In [9]:
rating_map = {
    'Two': 2,
    'One': 1,
    'Five': 5,
    'Four': 4,
    'Three': 3
}

In [10]:
books['star_rating'] = books['star_rating'].apply(lambda x: x.strip())

In [11]:
books['star_rating'] =books['star_rating'].replace(rating_map)

In [12]:
books.corr()

,price,star_rating
price,1.000000,0.028166
star_rating,0.028166,1.000000


#### There is no strong correlation between price and star rating

In [13]:
books.groupby('star_rating').mean()['price']

star_rating
1    34.561195
2    34.810918
3    34.692020
4    36.093296
5    35.374490
Name: price, dtype: float64

#### Mean of book price by rating there is not any connection between star rating and corr

# Problem 2

In [14]:
def scraping_jobs(url):
    page = requests.get(url)
    response = TextResponse(body=page.text,url=url,encoding="utf-8")
    company_name = response.xpath("//p[@class='job_list_company_title']/text()").extract()
    job_name = response.xpath("//p[@class='font_bold']/text()").extract()
    base_url = "https://staff.am"
    urls = response.xpath("//div[@class='web_item_card hs_job_list_item']/a/@href").extract()
    job_url = [base_url + i for i in urls]
    deadline_css = response.css("div[class = 'job-inner job-list-deadline'] >p:not([class='job_location'])")
    deadlinee_ext =[i.css('::text').extract()[1] for i in deadline_css]
    deadline = [i.replace('\n'," ") for i in deadlinee_ext]
    loc_css = response.css("div[class = 'job-inner job-list-deadline'] >p[class='job_location']")
    loc_ext = [i.css("::text").extract()[1] for i in loc_css]
    location = [i.replace('\n',"").strip() for i in loc_ext]
    return pd.DataFrame({"company_name":company_name,"job_name":job_name,'job_url':job_url,"deadline":deadline,'location':location})


In [15]:
jobs = []
for i in range(1,100):
    cureent_page =scraping_jobs(url = f"https://staff.am/en/jobs?page={i}&per-page=50")
    if cureent_page.shape[0] == 0:
        break
    else:
        jobs.append(cureent_page)

In [16]:
jobs= pd.concat(jobs)
jobs

,company_name,job_name,job_url,deadline,location
0,SFL,Digital Marketing Specialist /Storyteller,https://staff.am/en/digital-marketing-speciali...,30 July 2020,Yerevan
1,Chemonics International Inc.,"Public Communications and Outreach Officer, Ar...",https://staff.am/en/public-communications-and-...,08 July 2020,Yerevan
2,MOA Communications,Senior React.js Developer,https://staff.am/en/senior-reactjs-developer-62,30 July 2020,Yerevan
3,Chemonics International Inc.,"Senior Governance Officer, Armenia Integrity P...",https://staff.am/en/senior-governance-officer,08 July 2020,Yerevan
4,Chemonics International Inc.,"Deputy Chief of Party, Armenia Integrity Project",https://staff.am/en/deputy-chief-of-party,08 July 2020,Yerevan
...,...,...,...,...,...
11,ServiceTitan,"Senior Software Engineer, Front-End",https://staff.am/en/sr-software-engineer-front...,03 July 2020,Yerevan
12,ServiceTitan,Senior Site Reliability Engineer,https://staff.am/en/senior-site-reliability-en...,15 July 2020,Yerevan
13,ServiceTitan,Lead Software Engineer,https://staff.am/en/lead-software-engineer,05 July 2020,Yerevan
14,ServiceTitan,QA Automation Engineer,https://staff.am/en/qa-automation-engineer-91,05 July 2020,Yerevan


In [17]:
jobs['company_name'].value_counts()

Digitain                         32
SoftConstruct                    23
ServiceTitan                     16
TeamViewer Armenia               12
Willing & Able Operations LLC    11
                                 ..
Brainstorm Technologies           1
Canar                             1
Out Of Office Design              1
Էվրի Դեյ ՓԲԸ                      1
Triple Consulting                 1
Name: company_name, Length: 226, dtype: int64

#### most of the job's are posted by Digitain(32 jobs),Softconstruct(23 jobs),ServiceTitan(16),Picsart(today deleted all jobs but today it had 23 jobs) etc.

In [21]:
jobs[jobs['job_name'].str.contains("Data")]

,company_name,job_name,job_url,deadline,location
26,buymie,Data analyst,https://staff.am/en/data-analyst-53,24 July 2020,Yerevan
0,Webb Fontaine Holding LLC,Data Engineer,https://staff.am/en/data-engineer-51,22 July 2020,Yerevan
35,ACBA-CREDIT AGRICOLE BANK CJSC,Junior Database Administrator,https://staff.am/en/junior-database-administra...,19 July 2020,Yerevan
9,Ucom CJSC,Senior Database Administrator,https://staff.am/en/senior-database-administra...,11 July 2020,Yerevan
33,Krisp,Data Specialist / QA Engineer,https://staff.am/en/data-specialist-qa-engineer-1,03 July 2020,Yerevan
47,BIVAC Armenia CJSC,Database Administrator,https://staff.am/en/database-administrator-83,02 July 2020,Yerevan
9,DISQO,"Engineering Manager, Java Microservices & Data",https://staff.am/en/engineering-manager-java-m...,03 July 2020,Yerevan


#### Job names which contain "Data" word

In [22]:
jobs[jobs['job_name'].str.contains("Data")]['job_name'].count()

7

#### count of job names which contain 'Data' word

In [23]:
jobs.to_csv('jobs.csv', index=False)